# Promedio RMS de $Ro = \left|\frac{\zeta}{f}\right|$ en función del tiempo

In [1]:
import gc
import warnings
warnings.filterwarnings('ignore')

In [2]:
import imageio
import numpy as np
from scipy import signal
from pathlib import Path
from itertools import product
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from xmitgcm import open_mdsdataset
from datetime import date,datetime,timedelta

In [3]:
from spectral_analysis.preprocess.common_vars import OUT_IMGS_FOLDER,MODEL_FOLDER,LUIGI_OUT_FOLDER
from common_vars.time_slices import idx_t
from common_vars.regions import ids_regions

In [4]:
def fig2img(fig):
    fig.canvas.draw()
    image = np.frombuffer(fig.canvas.tostring_rgb(),dtype='uint8')
    image  = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    return image

In [5]:
def savefig(f,*args,**kvargs):
    fig = f(*args,**kvargs)
    image = fig2img(fig)
    plt.close(fig=fig)
    return image

In [6]:
grid_ds = open_mdsdataset(MODEL_FOLDER+'/LLC2160/grid/', read_grid=True, iters=None, default_dtype=np.float32, geometry='llc')
grid_ds

<xarray.Dataset>
Dimensions:  (face: 13, i: 2160, i_g: 2160, j: 2160, j_g: 2160, k: 90, k_l: 90, k_p1: 91, k_u: 90)
Coordinates:
  * i        (i) int64 0 1 2 3 4 5 6 7 ... 2153 2154 2155 2156 2157 2158 2159
  * i_g      (i_g) int64 0 1 2 3 4 5 6 7 ... 2153 2154 2155 2156 2157 2158 2159
  * j        (j) int64 0 1 2 3 4 5 6 7 ... 2153 2154 2155 2156 2157 2158 2159
  * j_g      (j_g) int64 0 1 2 3 4 5 6 7 ... 2153 2154 2155 2156 2157 2158 2159
  * k        (k) int64 0 1 2 3 4 5 6 7 8 9 10 ... 80 81 82 83 84 85 86 87 88 89
  * k_u      (k_u) int64 0 1 2 3 4 5 6 7 8 9 ... 80 81 82 83 84 85 86 87 88 89
  * k_l      (k_l) int64 0 1 2 3 4 5 6 7 8 9 ... 80 81 82 83 84 85 86 87 88 89
  * k_p1     (k_p1) int64 0 1 2 3 4 5 6 7 8 9 ... 81 82 83 84 85 86 87 88 89 90
  * face     (face) int64 0 1 2 3 4 5 6 7 8 9 10 11 12
    XC       (face, j, i) >f4 dask.array<chunksize=(1, 2160, 2160), meta=np.ndarray>
    YC       (face, j, i) >f4 dask.array<chunksize=(1, 2160, 2160), meta=np.ndarray>
    XG      

In [ ]:
LON_ = grid_ds.XG.values
LAT_ = grid_ds.YG.values
DEPTH_ = grid_ds.Depth.values

In [7]:
uv_fname_fmt_days = LUIGI_OUT_FOLDER+"/Datasets/{0}/days/{1}0_{2:05d}.txt"
uv_fname_fmt_hours = LUIGI_OUT_FOLDER+"/Datasets_compressed/{0}/hours/{1}0_{2:05d}.npz"
def uv4idt(region_id,t_idx,t_res="days"):
	if t_res=="days":
		U_ = np.loadtxt(uv_fname_fmt_days.format(region_id,"U",t_idx))
		V_ = np.loadtxt(uv_fname_fmt_days.format(region_id,"V",t_idx))
	else:
		U_ = np.load(uv_fname_fmt_hours.format(region_id,"U",t_idx))["uv"]
		V_ = np.load(uv_fname_fmt_hours.format(region_id,"V",t_idx))["uv"]
	return U_,V_

In [8]:
def UV4id(id,season,t_res="days",t_firstaxis=False):
	time = idx_t[t_res][season]
	
	for idx,t in enumerate(time):
		U_,V_ = uv4idt(id,t,t_res)
		if idx==0:
			#print(id,t)
			shape_uv = U_.shape
			#print("UV shape",shape_uv)
			shape = (shape_uv[0],shape_uv[1],len(time))
			U = np.zeros(shape)
			V = np.zeros(shape)
		U[:,:,idx] = U_
		V[:,:,idx] = V_

	if t_firstaxis:
		U = np.moveaxis(U,-1,0)
		V = np.moveaxis(V,-1,0)

	return U,V

In [9]:
dxdy_fname_fmt = LUIGI_OUT_FOLDER+"/Datasets/{}/days/{}.txt"
def dxdyf4id(region_id):
    dx_ = np.loadtxt(dxdy_fname_fmt.format(region_id,"dx"))
    dy_ = np.loadtxt(dxdy_fname_fmt.format(region_id,"dy"))
    f_ = np.loadtxt(dxdy_fname_fmt.format(region_id,"f"))
    
    return dx_,dy_,f_

def lonlat4id(region_id):
    lon = np.loadtxt(dxdy_fname_fmt.format(region_id,"lon"))
    lat = np.loadtxt(dxdy_fname_fmt.format(region_id,"lat"))
    return lon,lat

In [10]:
def rv_(u,v,dx,dy):
    return (np.gradient(v,axis=-1,edge_order=2)/dx) - (np.gradient(u,axis=-2,edge_order=2)/dy)

In [11]:
def getBathymetry(area_box):
    print("Bathymetry for box",area_box)
    min_lon,max_lon,min_lat,max_lat = area_box
    mask_lonlat_f = np.where((LON_>=min_lon) & (LON_<=max_lon) & (LAT_>=min_lat) & (LAT_<=max_lat))
    faces = set(mask_lonlat_f[0])
    print("faces",faces)
    LON_area = [None]*len(faces)
    LAT_area = [None]*len(faces)
    DEPTH_area = [None]*len(faces)
    for i,face in enumerate(faces):
        LON_f = LON_[face,:,:]
        LAT_f = LAT_[face,:,:]
        mask_lonlat = np.where((LON_f>=min_lon) & (LON_f<=max_lon) & (LAT_f>=min_lat) & (LAT_f<=max_lat))
        min_i,max_i = min(mask_lonlat[0]),max(mask_lonlat[0])
        min_j,max_j = min(mask_lonlat[1]),max(mask_lonlat[1])
        print(face,min_i,max_i,min_j,max_j)
        LON_area[i] = LON_f[min_i:max_i,min_j:max_j]
        LAT_area[i] = LAT_f[min_i:max_i,min_j:max_j]
        DEPTH_area[i] = DEPTH_[face,min_i:max_i,min_j:max_j]
    return LON_area,LAT_area,DEPTH_area

In [12]:
def plotBathymetry(area_box):
    LON,LAT,DEPTH = getBathymetry(area_box)
    print("depth len",len(DEPTH))
    for i in range(len(DEPTH)):
        LON_f = LON[i]
        LAT_f = LAT[i]
        DEPTH_f = DEPTH[i]
        print(LON_f.shape,LAT_f.shape,DEPTH_f.shape)
        #plt.pcolormesh(LON_f,LAT_f,DEPTH_f,cmap=plt.cm.Pastel1,norm=norm_bath)
        cs = plt.contour(LON_f,LAT_f,DEPTH_f,levels=[0,200,1000],cmap=plt.cm.Greys_r,extend="max",zorder=0)
        plt.clabel(cs, inline=True, fontsize='large',fmt="%d")

In [13]:
margin = 6
firstday = datetime(2012,1,1)

def plot_ro_region_t(current,t_idx,t_vec,t_res="days",norm=False,absolute=False):
	rv_range = 1 if norm else 0.8
	rv_type = "Normalized " if norm else ""
	o_fmt = "" if norm else " $[10^{-4} s^{-1}]$"
	dt_days = 1 if t_res=="days" else 24
	divnorm_ro = colors.DivergingNorm(vmin=-rv_range, vcenter=0, vmax=rv_range)
	
	#fig, ax = plt.subplots(1,2,figsize=(20,10))
	fig = plt.figure(figsize=(10,10))
	#ax = ax.flat[:]
	t_ = t_vec[t_idx]
	t__ = t_ - idx_t[t_res]["JFM"][0]
	t_delta = timedelta(hours=t__) if t_res=="hours" else timedelta(days=t__)
	t_label = (firstday + t_delta).date()
	#t_label = [t__-idx_t[t_res]["JFM"][0]+1 for t__ in t_vec]
	print(t_idx+1,"/",len(t_vec))
	lon_min,lat_min,lon_max,lat_max = (180,90,-180,-90)

	for r_id in ids_regions[current]:
		#Ro_vec = Ro_dict[r_id]
		print("U,V for {} ({})".format(r_id,t_res))
		U,V = uv4idt(r_id,t_,t_res)
		DX,DY,F = dxdyf4id(r_id)
		norm_rv = F if norm else 1e-4
		rv_sum = F if absolute else 0
		Ro = (rv_(U,V,DX,DY)+rv_sum)/norm_rv
		#Ro_vec[t_idx] = np.sqrt(np.mean(Ro**2))
		LON,LAT = lonlat4id(r_id)
		lon_min = min(lon_min,np.min(LON))
		lat_min = min(lat_min,np.min(LAT))
		lon_max = max(lon_max,np.max(LON))
		lat_max = max(lat_max,np.max(LAT))
		#plt.sca(ax[0])
		plt.pcolormesh(LON,LAT,Ro,norm=divnorm_ro,cmap=plt.cm.RdBu,zorder=10)
		#plt.sca(ax[1])
		#plt.plot(t_label,Ro_vec,label="{0:d} @ {1:0.1f},{2:0.1f}".format(r_id,np.mean(LON),np.mean(LAT)))
	#plt.sca(ax[0])
	plt.colorbar()
	box = (lon_min,lon_max+margin,lat_min,lat_max)
	plotBathymetry(box)
	plt.xlabel("Longitude",fontsize='large')
	plt.ylabel("Latitude",fontsize='large')
	plt.title(t_label,fontsize='x-large')
	#plt.sca(ax[1])
	#plt.xlim([t_label[0],t_label[-1]])
	#plt.xlabel("Year{}".format(t_res),fontsize='large')
	#plt.ylabel("RMS(Ro)",fontsize='large')
	#plt.title("RMS(Ro) by region - {}".format(current),fontsize='x-large')
	#plt.legend()
	plt.axis("equal")

	fig.suptitle("{}Relative Vorticity{}".format(rv_type,o_fmt),fontsize='xx-large')
	return fig#,ax

In [ ]:
t_r = "days"
fps = 3 if t_r == "days" else 6
for current,ids in ids_regions.items():
	if current=="California":
		for season in ["ASO","JFM"]:
			times = idx_t[t_r][season]
			n_times = len(times)
			#o_d = {r_id:[None]*n_times for r_id in ids}
			print(current,season)
			fname_gif = '{}/rms_ro_all_{}_{}.gif'.format(OUT_IMGS_FOLDER,current,season)
			Path(fname_gif).parent.mkdir(parents=True, exist_ok=True)
			imageio.mimsave(fname_gif, 
							(savefig(plot_ro_region_t,current,idx_t,times,t_r,norm=False,absolute=False) for idx_t in range(n_times)), 
							 fps=fps,loop=10)
			print("Saved!",fname_gif)
		gc.collect()

## Graficando RMS(Ro)

In [14]:
def getRoRms(r_id,season,t_res="days",norm=False,absolute=False):
	U,V = UV4id(r_id,season,t_res=t_res,t_firstaxis=True)
	DX,DY,F = dxdyf4id(r_id)
	norm_rv = F if norm else 1e-4
	sum_rv = F if absolute else 0
	Ro = (rv_(U,V,DX,DY)+sum_rv)/norm_rv
	#return np.sqrt(np.mean(Ro**2,axis=(1,2)))
	return np.std(Ro,axis=(1,2))

In [15]:
prop_cycle = plt.rcParams['axes.prop_cycle']

def roRmsByCurrent(current,season,t_res="days",norm=True,absolute=False):
	times = np.array(idx_t[t_res][season])
	n_times = len(times)
	n_ids = len(ids_regions[current])
	allRoRms = [getRoRms(r_id,season,t_res,norm,absolute) for r_id in ids_regions[current]]
	colors = prop_cycle.by_key()['color']
	var = "Ro" if norm else "RV"
	o_fmt = "" if norm else "\n$[10^{-4} s^{-1}]$"
	
	if t_res=="days":
		f_ticks_lab = [2,7,15,21,30,40]
		day_dt = 1
	else:
		f_ticks_lab = [h/24 for h in [6,12,24,7*24,15*24,30*24]]
		day_dt = 24
		
	t_label = [(t__-idx_t[t_res]["JFM"][0]+1)/day_dt for t__ in times]
	f = np.fft.rfftfreq(n_times,1) # T=1 {t_days}: intervalo de muestreo
		
	f_ticks = [1/(24*D) for D in f_ticks_lab]
	fig = plt.figure(figsize=(12,8),constrained_layout=True)
	gs = fig.add_gridspec(n_ids,2)
	
	axT = []
	axF = []
	maxY_t = 0
	minY_t = 1
	maxY_f = 0
	for i,r_id,ro_rms in zip(range(n_ids),ids_regions[current],allRoRms):
		_,LAT = lonlat4id(r_id)
		lat = np.median(LAT)
		lbl = "{0} @ {1:0.1f}".format(r_id,lat)
		color = colors[i]
		#print("color",color)
		# Time domain
		ax0 = fig.add_subplot(gs[i,0])
		axT.append(ax0)
		plt.sca(ax0)
		maxY_t = max(maxY_t,np.max(ro_rms))
		minY_t = min(minY_t,np.min(ro_rms))
		plt.plot(t_label,ro_rms,label=lbl,color=color)
		plt.ylabel("Std({}){}".format(var,o_fmt),size="x-large")
		# Frequency domain
		ax1 = fig.add_subplot(gs[i,1])
		axF.append(ax1)
		plt.sca(ax1)
		ro_rms_det = signal.detrend(ro_rms,type='linear')
		rorms_f = np.abs(np.fft.rfft(ro_rms_det))
		maxY_f = max(maxY_f,np.max(rorms_f))
		plt.loglog(f,rorms_f,label=r_id,color=color)
		plt.xticks(ticks=f_ticks,labels=f_ticks_lab,size="large")
		print(r_id,minY_t,maxY_t,maxY_f)
		if i==n_ids-1:
			ax0.set_xlabel("Yeardays",size="x-large")
			ax1.set_xlabel("Period in days",size="x-large")
		if i==0:
			ax0.set_title("Time domain\n"+lbl)
			ax1.set_title("FFT \n"+lbl)
		else:
			ax0.set_title(lbl)
			ax1.set_title(lbl)
	plt.suptitle("{} / {}".format(current,season),size="xx-large")
	for axt,axf in zip(axT,axF):
		axt.set_ylim([minY_t,maxY_t])
		axf.set_ylim([1e-1,maxY_f])
	plt.show()	#print("Box",box)

In [ ]:
for current,season in product(ids_regions.keys(),["JFM","ASO"]):
	print(current,season)
	roRmsByCurrent(current,season,"hours")

### * Extra: Creando animación de RV para una región cualquiera

In [18]:
def plot_rv(rv,lon,lat):
	std = np.std(rv)
	divnorm_rv = colors.DivergingNorm(vmin=-1, vcenter=0, vmax=1)
	fig = plt.figure(figsize=(10,10))
	plt.pcolor(lon,lat,rv,norm=divnorm_rv,cmap=plt.cm.RdBu)
	plt.title("std(RV)={0:0.3f}".format(std),size="xx-large")
	return fig

def anim_rv(r_id,season):
	LON,LAT = lonlat4id(r_id)
	U,V = UV4id(r_id,season,t_res="days",t_firstaxis=True)
	DX,DY,F = dxdyf4id(r_id)
	nt,_,_ = U.shape
	rv_norm = rv_(U,V,DX,DY)/F
	#
	fname_gif = '{}/rv_{}_{}.gif'.format(OUT_IMGS_FOLDER,r_id,season)
	Path(fname_gif).parent.mkdir(parents=True, exist_ok=True)
	imageio.mimsave(fname_gif, 
					(savefig(plot_rv,rv_norm[i,:,:],LON,LAT) for i in range(nt)), 
					 fps=3,loop=10)
	print("Saved!",fname_gif)

In [19]:
anim_rv(730,'JFM')

Saved! /home/antonio/GoogleDrive/Tesis/data/output_images/rv_730_JFM.gif
